In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


In [ ]:
events_df.columns

## Cantidad eventos total

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Quincenas

In [9]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [10]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [11]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [12]:
features_df = features_df.join(df, on='person')

## Cant ev indiv

In [13]:
df = events_df.groupby('person').event.value_counts()

In [14]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [15]:
df = df.drop(columns=['lead'])

In [16]:
features_df = features_df.join(df, on='person')

In [17]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## Dia semana

In [123]:
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [124]:
ct = pd.crosstab(events_df.person, events_df.weekday)
ct = ct.astype('bool')

In [125]:
from mlxtend.frequent_patterns import apriori 

frequent_itemsets = apriori(ct, min_support=0.4, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.414098,(Thursday),1
1,0.404466,(Tuesday),1
2,0.416982,(Wednesday),1


In [126]:
result = columna_por_persona_new(ct)

In [127]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38829

In [128]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38829

In [129]:
d = {'person': lista_personas, 'weekday_support': lista_support}
df = pd.DataFrame(data=d)

In [130]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,q9,q10,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,url_support,weekday_support
0,4886f805,9,False,True,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,0.0
1,ad93850f,65,True,True,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,0.010064,0.0
2,0297fc1e,567,True,True,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,0.000000,0.0


## Supports 

In [77]:
## url

ct = pd.crosstab(events_df.person, events_df.url)
ct = ct.astype('bool')

In [78]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.518247,(/),1
1,0.080857,(/comprar/iphone),1
2,0.012823,(/comprar/iphone/),1
3,0.099386,(/comprar/iphone/6),1
4,0.016961,(/comprar/iphone/6-plus),1


In [79]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [80]:
result = columna_por_persona(ct)

In [81]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support   

In [82]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value 

In [83]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

31896

In [84]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas 

In [85]:
lista_personas = lista_personas_f(result)
len(lista_personas)

31896

In [86]:
d = {'person': lista_personas, 'url_support': lista_support}
df = pd.DataFrame(data=d)

In [87]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,q1,q2,q3,q4,q5,q6,q7,q8,...,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,url_support
0,4886f805,9,False,False,False,False,False,False,False,False,...,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN
1,ad93850f,65,False,False,False,False,False,False,False,False,...,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,0.010064
2,0297fc1e,567,True,True,True,True,True,True,True,True,...,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,0.000000


In [ ]:
#sku (no, mucho por procesar)

ct = pd.crosstab(events_df.person, events_df.sku)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona(ct)
result.shape

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'weekday_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [132]:
# model

ct = pd.crosstab(events_df.person, events_df.model)
ct = ct.astype('bool')

In [133]:
ct.shape

(38256, 208)

In [134]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.1, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.132136,(Samsung Galaxy J5),1
1,0.105369,(Samsung Galaxy J7 Prime),1
2,0.105395,(Samsung Galaxy S6 Edge),1
3,0.127614,(Samsung Galaxy S6 Flat),1
4,0.157256,(Samsung Galaxy S7),1


In [135]:
result = columna_por_persona(ct)

In [136]:
len(result)

38256

In [137]:
features_df.shape

(38829, 16)

In [138]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38256

In [139]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38256

In [140]:
d = {'person': lista_personas, 'model_support': lista_support}
df = pd.DataFrame(data=d)

In [141]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,q9,q10,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,url_support,weekday_support,model_support
0,4886f805,9,False,True,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,0.0,0.105369
1,ad93850f,65,True,True,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,0.010064,0.0,0.000000
2,0297fc1e,567,True,True,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,0.000000,0.0,0.000000


In [142]:
result = columna_por_persona_new(ct)

In [143]:
len(result)

38256

In [144]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38256

In [145]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38256

In [146]:
d = {'person': lista_personas, 'model_ind_support': lista_support}
df = pd.DataFrame(data=d)

In [147]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,q9,q10,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,url_support,weekday_support,model_support,model_ind_support
0,4886f805,9,False,True,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,0.0,0.105369,0.105369
1,ad93850f,65,True,True,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,0.010064,0.0,0.000000,0.000000
2,0297fc1e,567,True,True,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,0.000000,0.0,0.000000,0.142095


In [148]:
# Condition

ct = pd.crosstab(events_df.person, events_df.condition)
ct = ct.astype('bool')

In [149]:
frequent_itemsets = apriori(ct, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.881935,(Bom),1
1,0.634013,(Excelente),1
2,0.660183,(Muito Bom),1


In [150]:
result = columna_por_persona_new(ct)

In [151]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38250

In [152]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38250

In [153]:
d = {'person': lista_personas, 'condition_support': lista_support}
df = pd.DataFrame(data=d)

In [154]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,q9,q10,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,url_support,weekday_support,model_support,model_ind_support,condition_support
0,4886f805,9,False,True,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,0.0,0.105369,0.105369,0.634013
1,ad93850f,65,True,True,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,0.010064,0.0,0.000000,0.000000,0.881935
2,0297fc1e,567,True,True,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,0.000000,0.0,0.000000,0.142095,0.881935


In [155]:
# Storage

ct = pd.crosstab(events_df.person, events_df.storage)
ct = ct.astype('bool')

In [156]:
frequent_itemsets = apriori(ct, min_support=0.5, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

,support,itemsets,length
0,0.693281,(16GB),1
1,0.629935,(32GB),1
2,0.509333,(64GB),1


In [157]:
result = columna_por_persona_new(ct)

In [158]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

38250

In [159]:
lista_personas = lista_personas_f(result)
len(lista_personas)

38250

In [160]:
d = {'person': lista_personas, 'storage_support': lista_support}
df = pd.DataFrame(data=d)

In [161]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

,person,cant_eventos_totales,q9,q10,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,url_support,weekday_support,model_support,model_ind_support,condition_support,storage_support
0,4886f805,9,False,True,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,4.0,1.0,NaN,0.0,0.105369,0.105369,0.634013,0.629935
1,ad93850f,65,True,True,10.0,15.0,1.0,NaN,7.0,7.0,NaN,NaN,20.0,5.0,0.010064,0.0,0.000000,0.000000,0.881935,0.000000
2,0297fc1e,567,True,True,29.0,4.0,7.0,NaN,21.0,NaN,6.0,NaN,404.0,95.0,0.000000,0.0,0.000000,0.142095,0.881935,0.000000


In [ ]:
#color

ct = pd.crosstab(events_df.person, events_df.color)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.2, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [49]:
def columna_por_persona_new(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        num_max = 0
        lista = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > num_max :
                value_max = value
                num_max = crosstab.loc[person, value]
        lista.append(value_max)
        values = frozenset( lista ) 
        diccionario[person] = values                
    return diccionario

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
len(result)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'color_3_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

In [ ]:
features_df = features_df.drop(columns=['color_2_support'])

In [ ]:
#Channel

ct = pd.crosstab(events_df.person, events_df.channel)
ct = ct.astype('bool')

In [ ]:
frequent_itemsets = apriori(ct, min_support=0.25, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona_new(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'channel_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Labels y separacion de df

In [18]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [204]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 15)

In [205]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 14)

In [206]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [207]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [208]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## XGBoost

In [24]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [25]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


In [26]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3, 0.5],
        'colsample_bylevel': [0.7, 0.8, 0.9],
        'max_depth': [3, 4, 5, 7, 8, 9, 11, 13, 15],
        'base_score': [0.1, 0.2, 0.3],
        'n_estimators': [35, 40, 50, 60, 70, 80, 90],
        'booster':['gbtree', 'dart']
        }

In [27]:
xg_gs = xgb.XGBRegressor(objective ='binary:logistic',
                        learning_rate = 0.2,
                        alpha = 20, eval_metric='auc')

In [167]:
folds = 10
param_comb = 7

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xg_gs, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, 
                                   cv=skf.split(train_df.drop(columns=['person']),labels), verbose=3, random_state=1001,return_train_score=True)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_df.drop(columns=['person']), labels)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   15.9s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:   27.8s finished



 Time taken: 0 hours 0 minutes and 30.25 seconds.


In [168]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)

print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([3.35700545, 1.77604554, 0.51318429, 0.28583207, 1.25224235,
       0.47593901, 1.77910519]), 'std_fit_time': array([0.02642   , 0.07050221, 0.0177381 , 0.00356556, 0.02421645,
       0.01444531, 0.16146543]), 'mean_score_time': array([0.01579709, 0.01008155, 0.00894756, 0.01084344, 0.01010993,
       0.01307757, 0.01099846]), 'std_score_time': array([0.00064821, 0.0001877 , 0.00031957, 0.00165083, 0.0009727 ,
       0.00039023, 0.00161867]), 'param_n_estimators': masked_array(data=[70, 70, 35, 40, 70, 50, 70],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 1, 10, 1, 5, 5, 1],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[5, 11, 4, 4, 5, 7, 15],
             mask=[False, False, False, False, False, False, False],
       

In [169]:
preds_grid_search = random_search.predict(X_test)

In [183]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds_grid_search)

0.9309044085948314

In [32]:
#Parametros con los que obtuvimos 0.82

xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [180]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [181]:
preds = xg_reg.predict(X_test)

In [182]:
roc_auc_score(y_test,preds)

0.8360098991608079

## Importancia con Random Forest

In [211]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
to_predict.fillna(0, inplace=True)

In [212]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [213]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [214]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [215]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.7477942439653692

In [216]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cant_eventos_totales Importance: 0.14
Variable: cant_vp              Importance: 0.12
Variable: cant_ch              Importance: 0.1
Variable: cant_bl              Importance: 0.09
Variable: cant_ach             Importance: 0.08
Variable: cant_gl              Importance: 0.08
Variable: cant_vs              Importance: 0.08
Variable: cant_seh             Importance: 0.07
Variable: cant_s_p             Importance: 0.07
Variable: url_support          Importance: 0.06
Variable: model_ind_support    Importance: 0.05
Variable: weekday_support      Importance: 0.03
Variable: storage_support      Importance: 0.03


In [131]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'q9', 'q10', 'cant_ach', 'cant_bl',
       'cant_ch', 'cant_cv', 'cant_gl', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'url_support', 'weekday_support'],
      dtype='object')

In [203]:
#features_df = features_df.drop(columns=['condition_support', 'model_support', 'cant_sp', 'cant_cv', 'q10', 'q9'])

In [192]:
preds_rf = rf.predict(to_predict.drop(columns=['person']))

In [193]:
preds_rf

array([0.        , 0.055     , 0.        , ..., 0.0179    , 0.18805   ,
       0.09168333])

## Prediccion

In [184]:
preds_posta = random_search.predict(to_predict.drop(columns=['person']))

In [185]:
preds_posta

array([0.00440792, 0.02441191, 0.00882648, ..., 0.11408028, 0.10112598,
       0.12690426], dtype=float32)

In [194]:
#encamble

ensamble = predictions*0.2 + preds_grid_search*0.8
roc_auc_score(y_test, ensamble)

0.915569461056494

In [195]:
ensamble = preds_rf*0.2 + preds_posta*0.8
ensamble

array([0.00352634, 0.03052953, 0.00706118, ..., 0.09484423, 0.11851078,
       0.11986008])

## Genero CSV a publicar

In [196]:
to_publish = pd.DataFrame()

In [197]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [198]:
to_publish['label'] = ensamble
to_publish.shape

(19415, 2)

In [199]:
to_publish.head()

,person,label
0,4886f805,0.003526
1,0297fc1e,0.030530
2,2d681dd8,0.007061
3,cccea85e,0.021187
4,4c8a8b93,0.039840


In [200]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [201]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [202]:
to_publish.to_csv('2_12_3.csv', index=False)